In [ ]:
pip install numpy==1.26.4 tensorflow opencv-python scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np

class SnakeBreedDetector:
    def __init__(self, input_shape=(224, 224, 3), data_dir=None, batch_size=32):
        self.input_shape = input_shape
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.class_names = self._get_class_names() if data_dir else None
        self.num_classes = len(self.class_names) if self.class_names else 0
        self.model = self._build_model()
        
    def _get_class_names(self):
        """Extract class names from subfolder names inside data_dir."""
        # Sort to keep order consistent.
        return sorted(os.listdir(self.data_dir))
    
    def _build_model(self):
        model = models.Sequential([
            # Convolutional layers with BatchNormalization for faster convergence.
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def preprocess_image(self, image_path):
        img = cv2.imread(image_path)
        img = cv2.resize(img, (self.input_shape[0], self.input_shape[1]))
        img = img / 255.0  # Normalize pixel values.
        return np.expand_dims(img, axis=0)
    
    def train(self, epochs=50):
        # Use ImageDataGenerator with data augmentation and a validation split.
        datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2  # Reserve 20% of data for validation.
        )
        
        train_generator = datagen.flow_from_directory(
            self.data_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training'
        )
        
        validation_generator = datagen.flow_from_directory(
            self.data_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation'
        )
        
        # Callbacks for advanced training:
        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        checkpoint = callbacks.ModelCheckpoint('best_snake_detector.h5', monitor='val_loss', save_best_only=True)
        reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
        
        history = self.model.fit(
            train_generator,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=[early_stop, checkpoint, reduce_lr]
        )
        return history
    
    def predict(self, image_path):
        img = self.preprocess_image(image_path)
        predictions = self.model.predict(img)[0]
        predicted_class = np.argmax(predictions)
        predicted_species = self.class_names[predicted_class]
        confidence = np.max(predictions)
        return predicted_species, confidence
    
    def save_model(self, path):
        self.model.save(path)
    
    @classmethod
    def load_model(cls, path, class_names):
        detector = cls()
        detector.model = tf.keras.models.load_model(path)
        detector.class_names = class_names
        detector.num_classes = len(class_names)
        return detector

# Example usage
if __name__ == "__main__":
    # Provide the path to your dataset containing subfolders for each snake species.
    dataset_path = "C:\\Users\\RENTKAR\\Downloads\\Telegram Desktop\\snake_data"
    
    detector = SnakeBreedDetector(data_dir=dataset_path, batch_size=32)
    
    # Train the model (with early stopping, checkpointing, and learning rate reduction)
    detector.train(epochs=50)
    
    # Save the trained model.
    detector.save_model("snake_detector.h5")
    
    

In [ ]:
# Load the model for inference
detector = SnakeBreedDetector.load_model("snake_detector.h5", class_names=detector.class_names)

# Make a prediction
species, confidence = detector.predict("C:\\Users\\RENTKAR\\Pictures\\ScreenShots\\python sample testing.png")

print(f"Predicted species: {species}")
print(f"Confidence: {confidence:.2f}")
